In [ ]:
from google.colab import drive
import os
drive.mount('/gdrive', force_remount=True)

os.chdir('/gdrive/MyDrive/DL_model')
os.getcwd()

Mounted at /gdrive


'/gdrive/MyDrive/DL_model'

In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 학습 데이터 TXT파일 생성
th = ''
def get_data(theme, v=7):
  original = open("/gdrive/MyDrive/DL_model/txt_data/{}.txt".format(theme), 'r')
  train = open("/gdrive/MyDrive/DL_model/train.txt",'a')
  test = open("/gdrive/MyDrive/DL_model/test.txt",'a')
  original_l = original.readlines()

  global th
  th = theme
  for i,line in enumerate(original_l):
    if (i/len(original_l)*100>=v*10):
      test.write(line)
    else:
      train.write(line)

  train.close()
  test.close()
  original.close()

In [ ]:
import torch
import os
from ratsnlp.nlpbook.classification import ClassificationExample
from transformers import BertTokenizer
from ratsnlp import nlpbook
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
from ratsnlp.nlpbook.classification import ClassificationDataset
from torch.utils.data import DataLoader, RandomSampler
from transformers import BertConfig, BertForSequenceClassification
from ratsnlp.nlpbook.classification import ClassificationTask

# ['스팸','가볼만한곳','가족여행','우정여행','전통','체험','캠핑','관람','맛집','카페']
theme_list = ['가볼만한곳']
for tt in theme_list:
  get_data(tt)
  args = ClassificationTrainArguments(
      pretrained_model_name="beomi/kcbert-base",
      downstream_task_name="pair-classification",
      downstream_corpus_name="klue-nli",
      downstream_model_dir="/gdrive/MyDrive/DL_model/checkpoint-paircls2/"+tt,
      batch_size=32 if torch.cuda.is_available() else 4,
      learning_rate=5e-5,
      max_seq_length=50,
      epochs=3,
      tpu_cores=0 if torch.cuda.is_available() else 8,
      seed=7,
  )

  #시드 고정
  nlpbook.set_seed(args)

  #로거 설정
  nlpbook.set_logger(args)

  #토크나이저 선언
  tokenizer = BertTokenizer.from_pretrained(
      args.pretrained_model_name,
      do_lower_case=False,
  )

  #학습 데이터


  class TravelPlus_NLICorpus:

      def __init__(self):
          pass

      def get_examples(self, data_root_path, mode):
          data_fpath = os.path.join("/gdrive/MyDrive/DL_model/", f"{mode}.txt")
          lines = open(data_fpath, "r", encoding="utf-8").readlines()
          data = []
          for (i, line) in enumerate(lines):
              if i == 0:
                  continue
              tags, labels = line.split(',') # 태그 라벨 분리
              data.append(ClassificationExample(text_a=tags, label=labels.replace('\n',''))) # 엔터 전처리
          return data

      def get_labels(self):
          return ["1", "0"]

      @property
      def num_labels(self):
          return len(self.get_labels())
          
  # 말뭉치 다운
  # nlpbook.download_downstream_dataset(args)


  corpus = TravelPlus_NLICorpus()
  train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
  )


  train_dataloader = DataLoader(
      train_dataset,
      batch_size=args.batch_size,
      sampler=RandomSampler(train_dataset, replacement=False),
      collate_fn=nlpbook.data_collator,
      drop_last=False,
      num_workers=args.cpu_workers,
  )

  # 테스트 데이터 구축

  val_dataset = ClassificationDataset(
      args=args,
      corpus=corpus,
      tokenizer=tokenizer,
      mode="test",
  )
  val_dataloader = DataLoader(
      val_dataset,
      batch_size=args.batch_size,
      
      collate_fn=nlpbook.data_collator,
      drop_last=False,
      num_workers=args.cpu_workers,
  )

  # 모델 초기화

  pretrained_model_config = BertConfig.from_pretrained(
      args.pretrained_model_name,
      num_labels=corpus.num_labels,
  )

  model = BertForSequenceClassification.from_pretrained(
          args.pretrained_model_name,
          config=pretrained_model_config,
  )

  # 학습 준비
  task = ClassificationTask(model, args)

  trainer = nlpbook.get_trainer(args)

  # 학습
  trainer.fit(
      task,
      train_dataloaders=train_dataloader,
      val_dataloaders=val_dataloader,
  )

  # 모델 저장하기

  path = '/gdrive/MyDrive/DL_model/save_pt2/'
  torch.save(model, path + str(th) + '_model.pt')  # 전체 모델 저장

  torch.save(model.state_dict(), path + str(th) + '_state_dict.pt')  # 모델 객체의 state_dict 저장

  torch.save({
      'model': model.state_dict()
  },  path + str(th) + '_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

  print('------------------------------------{}완료------------------------------------'.format(str(th)))
  


INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/DL_model/checkpoint-paircls2/가볼만한곳', max_seq_length=50, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=4, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/MyDrive/DL_model/checkpoint-paircls2/가볼만한곳', max_seq_length=50, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epoch

set seed: 7


INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_train_BertTokenizer_50_klue-nli_pair-classification [took 1.594 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_train_BertTokenizer_50_klue-nli_pair-classification [took 1.594 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_test_BertTokenizer_50_klue-nli_pair-classification [took 1.549 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/klue-nli/cached_test_BertTokenizer_50_klue-nli_pair-classification [took 1.549 s]
Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight',

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

------------------------------------가볼만한곳완료------------------------------------
